In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI


In [3]:
## defining LLM
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash", convert_system_message_to_human=True)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001FA70A7A250>, default_metadata=(), convert_system_message_to_human=True, model_kwargs={})

In [4]:
llm.invoke([HumanMessage(content = "Hi, my name is Ash and I am a GenAI Engineer")])

c:\Users\ashut\OneDrive\Documents\study material\NLP LLM\Agentic AI\Chatbot_w_Memory\venv\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content="Hi Ash, nice to meet you! It's great to connect with a fellow GenAI Engineer. The field is incredibly exciting and rapidly evolving.\n\nWhat kind of GenAI projects are you working on? I'm always interested in learning about what others are doing in the space. Are you focusing on specific modalities (text, image, audio, etc.) or applications?\n\nLet me know if there's anything I can help you with, whether it's brainstorming ideas, sharing resources, or just discussing the latest trends.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-d01f920e-bec8-4015-9da7-a29469cf41a1-0', usage_metadata={'input_tokens': 13, 'output_tokens': 109, 'total_tokens': 122, 'input_token_details': {'cache_read': 0}})

In [6]:
llm.invoke(
   [
       HumanMessage(content="Hi, my name is Ash and I am a GenAI Engineer"),
       AIMessage(content="Hi Ash, nice to meet you! It's great to connect with a fellow GenAI Engineer. The field is incredibly exciting and rapidly evolving.\n\nWhat kind of GenAI projects are you working on? I'm always interested in learning about what others are doing in the space. Are you focusing on specific modalities (text, image, audio, etc.) or applications?\n\nLet me know if there's anything I can help you with, whether it's brainstorming ideas, sharing resources, or just discussing the latest trends."),
       HumanMessage(content="Hey what is my name and what do I do?")
   ] 
)

c:\Users\ashut\OneDrive\Documents\study material\NLP LLM\Agentic AI\Chatbot_w_Memory\venv\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content='Your name is Ash, and you are a GenAI Engineer.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-de7d729f-0bfb-4662-9fab-36f176e9e202-0', usage_metadata={'input_tokens': 132, 'output_tokens': 14, 'total_tokens': 146, 'input_token_details': {'cache_read': 0}})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep the track of inputs and outputs of the model, and store them in some datasore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    """ Get the chat message history for a given session ID.
    If the session ID does not exist, create a new one.
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## Create a RunnableWithMessageHistory instance
## This will allow you to use the LLM with message history.
## The get_session_history function will be used to retrieve the message history for a given session ID.
with_message_history = RunnableWithMessageHistory(llm, get_session_history)


In [12]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
## Invoke the LLM with message history
## The session ID is passed in the config dictionary.
response = with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Ash and I am a GenAI Engineer")],
    config=config
)

c:\Users\ashut\OneDrive\Documents\study material\NLP LLM\Agentic AI\Chatbot_w_Memory\venv\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [14]:
response.content

"Hi Ash! It's great to meet you. As a GenAI Engineer, you're working in a very exciting and rapidly evolving field. What are some of the things you enjoy most about your work? What kind of projects are you typically involved in? I'm curious to learn more about your experience!"

In [ ]:
## We can see that the message history is being used to keep track of the conversation.
with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)

c:\Users\ashut\OneDrive\Documents\study material\NLP LLM\Agentic AI\Chatbot_w_Memory\venv\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


AIMessage(content='Your name is Ash.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-59fd1d6f-4f9a-4f1f-b8df-eb0ecba46e71-0', usage_metadata={'input_tokens': 81, 'output_tokens': 6, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}})

In [ ]:
## now we will change the session id to "chat2" and see if the message history is being used.
config2 = {"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config = config2
)

## We can see that the message history is not being used for the new session ID "chat2".
response.content


c:\Users\ashut\OneDrive\Documents\study material\NLP LLM\Agentic AI\Chatbot_w_Memory\venv\Lib\site-packages\langchain_google_genai\chat_models.py:390: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


"As a large language model, I don't have access to personal information. Therefore, I do not know your name. You have not provided it to me."